In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
train_df=pd.read_csv('./output/train_hypothesis_evidences.csv')
train_df

,hypothesis,evidences,label
0,Patients with cytologically confirmed breast c...,"[""Inclusion Criteria:"", "" Histologically or c...",Contradiction
1,Women with leptomeningeal metastases are unfor...,"["" Exclusion Criteria"", "" Patients with know...",Contradiction
2,Cohort 1 of NCT02513472 produced better Object...,"[""Outcome Measurement: "", "" Objective Respons...",Entailment
3,NCT01237327 and NCT00030823 do not have any ov...,"[""Inclusion Criteria:"", "" Previous participat...",Entailment
4,More than 2% of patients in NCT00129376 experi...,"[""Adverse Events 1:"", "" Acute Pharyngitis * 1...",Contradiction
...,...,...,...
715,NCT00696072 uses Overall Survival (OS) as outc...,"[""Outcome Measurement: "", "" Proportion of Par...",Contradiction
716,NCT00320541 recorded more cardiac related adve...,"[""Adverse Events 1:"", "" Total: 17/50 (34.00%)...",Contradiction
717,There were 2 cases of Angina in NCT02370238.,"[""Adverse Events 1:"", "" Total: 13/61 (21.31%)...",Contradiction
718,NCT00645333 records several central nervous sy...,"["" Autoimmune disorder *1/30 (3.33%)"", "" Imm...",Contradiction


In [3]:
hypothesis_lst=train_df['hypothesis'].values.tolist()
len(hypothesis_lst)

720

In [4]:
evidence_lst=train_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

720

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=train_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

720

In [6]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [7]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
# text_tok=AutoTokenizer.from_pretrained('bert-base-uncased')
# text_clf=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
#roberta-base
text_tok=AutoTokenizer.from_pretrained('roberta-base')
text_clf=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [9]:
training_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)
len(training_data)

tokenize done


12

In [10]:
class Model(nn.Module):
    def __init__(self,clf):
        super(Model, self).__init__()
        self.clf=clf
        self.loss=nn.CrossEntropyLoss()
    
    def forward(self, texts, labels, gpu=True):
        
        loss=self.loss(self.clf(**texts).logits, labels)
        
        return loss

In [11]:
model=Model(text_clf)

In [12]:
bat_s=32
l_rate=1e-5

training_data.batch_size=bat_s

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
total_epoch_num=10
for epoch in range(total_epoch_num):
    training_data.on_epoch_end()
    loss_sum=0.0
    loss_count=0
    for batch in range(len(training_data)):
        optimizer.zero_grad()
        batch_texts,batch_labels=training_data[batch]
        loss_count+=len(batch_texts['input_ids'])
        loss = model(
            batch_texts,batch_labels
        )
        print('epoch:',epoch,'batch:',batch,'loss:',loss.item(),end='\n' if batch==0 or batch+1==len(training_data) or (batch+1)%1000==0 else '\r')
        loss_sum += 1.0*loss.item()*len(batch_texts['input_ids'])
        loss.backward()
        optimizer.step()
    # model.clf.save_pretrained('./output/clf_models/bert-base-uncased_epoch_{}.pt'.format(format(epoch,'05d')))
    #roberta-base
    model.clf.save_pretrained('./output/clf_models/roberta-base_epoch_{}.pt'.format(format(epoch,'05d')))
_=model.cpu()

epoch: 0 batch: 0 loss: 0.6977713108062744
epoch: 0 batch: 22 loss: 0.7047663927078247
epoch: 1 batch: 0 loss: 0.6914390325546265
epoch: 1 batch: 22 loss: 0.6853869557380676
epoch: 2 batch: 0 loss: 0.6950498223304749
epoch: 2 batch: 22 loss: 0.6986944675445557
epoch: 3 batch: 0 loss: 0.6923381686210632
epoch: 3 batch: 22 loss: 0.7052376270294193
epoch: 4 batch: 0 loss: 0.672080934047699
epoch: 4 batch: 22 loss: 0.6746116280555725
epoch: 5 batch: 0 loss: 0.7033475041389465
epoch: 5 batch: 22 loss: 0.6757913231849671
epoch: 6 batch: 0 loss: 0.6580055356025696
epoch: 6 batch: 22 loss: 0.6612647771835327
epoch: 7 batch: 0 loss: 0.6295934915542603
epoch: 7 batch: 22 loss: 0.5723339915275574
epoch: 8 batch: 0 loss: 0.531297504901886
epoch: 8 batch: 22 loss: 0.5323455929756165
epoch: 9 batch: 0 loss: 0.4005983769893646
epoch: 9 batch: 22 loss: 0.52865719795227056
